In [1]:
from IPython.display import Javascript

In [2]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [3]:
%%javascript

require(['d3'], function (d3) {
    
    console.clear();
    
    $("#chart1").remove();
    element.append("<div id='chart1'></div>");
    $("#chart1").width("960px");
    $("#chart1").height("3000px");
    
    //function for limiting the max length of a string
    function truncate(str, max_length) {
        var strList = str.split(' ');
        if (strList.length > max_length) {
            strList = strList.slice(0, max_length);
            str = strList.join(' ');
        };
        return str;
    };
    
    var canvas = d3.select("#chart1");
    var mousePosition;
        
    var margin = {top: 20, right: 20, bottom: 30, left: 40};
    var width = 960 - margin.left - margin.right,
        height = 3000 - margin.top - margin.bottom;

    var xAxis_length = 450;
    
    var y_margin = 45;
    
    var c = d3.scale.category20();
    var tempColor;
    
    var start_time = 2000,
        end_time = 2016;
    
    var x = d3.scale.linear()
              .range([0, xAxis_length]);

    var xAxis = d3.svg.axis()
                  .scale(x)
                  .orient("top")
                  .ticks(17);
    
    var formatTime = d3.format("000000");
    xAxis.tickFormat(formatTime);
    
    x.domain([start_time, end_time]);
    
    var xScale = d3.scale.linear()
                   .domain([start_time, end_time])
                   .range([0, xAxis_length]);

    var svg = canvas.append("svg")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
                .append("g")
                .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
    
    var axisNodes = svg.append("g")
                       .attr("class", "x axis")
                       .attr("transform", "translate(0," + 0 + ")")
                       .call(xAxis);
    
    svg.selectAll(".axis line, .axis path")
       .style({"stroke": "Black", "fill": "none", "stroke-width": "1px"});
    
    svg.selectAll(".axis text")
       .style({"font-size": "10px"});
    
    svg.selectAll(".axis")
       .append("text")
       .attr("y", -2)
       .attr("x", xAxis_length + 35)
       .attr("class","title")
       .text("Words Appearing in All Time Slices")
       .style({"font-size": "18px"})
       .style("text-decoration", "underline");
    
    var tooltip = d3.select("#chart1").append("div")
                    .attr("class", "tooltip")
                    .attr("position", "absolute")
                    .style("opacity", 0)
                    .style("background-color", "Rgba(0,0,0,0.8)");
    
    d3.json("lda_vis.json", function(json_data) {
        
        var data = [];
        
        //transfer json strings to json objects
        for (var i = 0; i < json_data.length; i++) {
            var one_row;
            JSON.parse(json_data[i], function(k, v) {
                one_row = v;
                return one_row;
            });
            data.push(one_row);
        };
        
        var previousColorNodes = [];
        
        var popularity = [];
        for (var i = 0; i < data.length; i++) {
            var one_row = data[i]['topics'];
            for (var j = 0; j < one_row.length; j++) {
                popularity.push(one_row[j][2]);
            };
        };
        
        var rScale = d3.scale.linear()
                       .domain([d3.min(popularity, function(d) { return d; }), 
                                d3.max(popularity, function(d) { return d; })]) 
                       .range([5, 18]);
        
        var links = [];
        var nodes = [];
        
        //draw links between similar topics
        for (var i = 0; i < data.length; i++) {
            for (var j = 0; j < data[i]['topics'].length; j++) {
                
                var dataItem = data[i]['topics'][j];
                var dataSize = dataItem[2];
                
                if (dataSize != ' ') {
                    
                    nodes.push({
                        id: xScale(dataItem[0]) + "" + i*y_margin+20,
                        x: xScale(dataItem[0]),
                        y: i*y_margin+20
                    });
                    links.push([i + "" + j, {
                        source: xScale(dataItem[0]) + "" + i*y_margin+20,
                        target: null
                    }])                 
                }
                
                if (j + 1 < data[i]['topics'].length) {
                    var nextSize = data[i]['topics'][j + 1][2];
                    var nextId = data[i]['topics'][j + 1][1];
                    var nextItem = data[i]['topics'][j + 1];
                    
                    if  (nextSize != ' ') {
                        links.forEach(function(item) {
                            if (item[0] == i + "" + j) {
                                item[1].target = xScale(nextItem[0]) + "" + i*y_margin+20;
                            }
                        });
                    }
                    
                    if (nextSize == ' ') {
                        for (var m = 0; m < data.length; m++) {
                            
                            for (var n = 0; n < data[m]['topics'].length; n++) {
                                
                                if (data[m]['topics'][n][1] == nextId && data[m]['topics'][n][2] != ' ') {
                                    links.forEach(function(item) {
                                        if (item[0] == i + "" + j) {
                                            item[1].target = xScale(data[m]['topics'][n][0]) + "" + m*y_margin+20;
                                        }
                                    });
                                }
                            }
                        }
                    }
                }
            }
        }
        
        var filteredLinks = [];
        links.forEach(function(link) {
            if (link[1].target) {
                filteredLinks.push(link[1]);
            }
        });
        
        var graph = {
            "nodes": nodes,
            "links": filteredLinks
        };
            
        var nodeById = d3.map();
        
        graph.nodes.forEach(function(node) {
            nodeById.set(node.id, node);
        });
            
        graph.links.forEach(function(link) {
            link.source = nodeById.get(link.source);
            link.target = nodeById.get(link.target);
        });
        
        var forceLayout = d3.layout.force()
                            .links(graph.links);
        
        var link = svg.selectAll("lines")
                      .data(graph.links)
                      .enter()
                      .append("line")
                      .attr("x1", function(d) { return d.source.x; })
                      .attr("y1", function(d) { return d.source.y; })
                      .attr("x2", function(d) { return d.target.x; })
                      .attr("y2", function(d) { return d.target.y; })
                      .attr("class", "link")
                      .attr("stroke", "Grey")
                      .attr("stroke-width", "1.5px");
                
        data.forEach(function(item, index) {
            
            //draw nodes in the matrix
            var g = svg.append("g").attr("class","topic");

            var circles = g.selectAll("circle")
                           .data(item['topics'])
                           .enter()
                           .append("circle");

            var text = g.selectAll("text")
                        .data(item['topics'])
                        .enter()
                        .append("text");
            
            circles
                .attr("cx", function(d, i) { return xScale(d[0]); })
                .attr("cy", index * y_margin + 20)
                .attr("r", function(d) {
                    if (d[2] != ' ') { return rScale(d[2]); };
                })
                .attr("class", function(d) { return d[1]; })
                .style("fill", "66aa00")
                .on("click", function(d) {
                
                    $("#popup").remove();
                
                    var isMergedNode = false;
                    var previousPopup = document.getElementById("single-node-popup");
                
                    if (previousPopup) {
                        for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                            previousColorNodes[i].style.fill = previousColorNodes[i].getAttribute("fill-color");
                        }
                        previousColorNodes.length = 0;
                        previousColorNodes = [];
                        previousPopup.remove();
                    }
                
                    //create a single topic word box
                    var sameIdCircles = document.getElementsByClassName(d[1]);
                    if (sameIdCircles.length > 1) {
                        for (var i = sameIdCircles.length - 1; i >= 0; i--) {
                            var currentNode = sameIdCircles[i];
                            var previousNode = currentNode.previousSibling;
                            if(previousNode != null){
                                if (previousNode.__data__[2] != ' ') {
                                    previousColorNodes.push(previousNode);
                                }
                            }
                        }
                            
                        if (previousColorNodes.length > 1) {
                            isMergedNode = true;
                            for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                previousColorNodes[i].setAttribute("fill-color", previousColorNodes[i].style.fill);
                                previousColorNodes[i].style.fill = c(i);
                                previousColorNodes[i].setAttribute("click-fill-color", 
                                                                   previousColorNodes[i].style.fill);
                            }
                        } else {
                            previousColorNodes[0].setAttribute("fill-color", previousColorNodes[0].style.fill);
                        }
                    }      
                        
                    var singleNodeBarDataset = d[3].sort(function(a, b) {
                        if (a[0] < b[0]) {
                            return 1;
                        }
                        if (a[0] > b[0]) {
                            return -1;
                        }
                        return 0;
                    });
                        
                    var barXScale = d3.scale.linear()
                                      .domain([0, d3.max(singleNodeBarDataset, function(d) {
                                            return d[0];
                                      })])
                                      .range([0,90]);
                
                    mousePosition = d3.mouse(canvas.node()); 
                                         
                    var popup = canvas.append("div")
                                      .attr("id", "single-node-popup")
                                      .style("position", "absolute")
                                      .style("left", (mousePosition[0] + 100) + "px")
                                      .style("top", (mousePosition[1] + 10) + "px")
                                      .style("background-color", "Rgba(0,0,0,0.8)")
                                      .style("padding", "5px");
                
                    popup.append("p") 
                         .style("font-size", "12px") 
                         .style("text-decoration", "underline")
                         .style("color", "SeaShell")
                         .style("padding", "5px")
                         .text("Word Frequency");
                        
                    var singlePopup = d3.select("#single-node-popup")
                                        .append("svg")
                                        .style("width", "100px")
                                        .style("height", "250px")
                                        .style("padding", "5px")
                                        .on("click", function() { 
                                            var previousPopup = document.getElementById("single-node-popup");
                                            if (previousPopup) {
                                                for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                                    previousColorNodes[i].style.fill = previousColorNodes[i]
                                                                                       .getAttribute("fill-color");
                                                }
                                                previousColorNodes.length = 0;
                                                previousColorNodes = [];
                                                previousPopup.remove();
                                            }
                                        });
                    
                    singlePopup.selectAll("rect")
                               .data(singleNodeBarDataset)
                               .enter()
                               .append("rect")
                               .attr("x", 0)
                               .attr("y", function(d, i) { return (i * 25); })
                               .attr("height", "20")
                               .attr("width", function(d) { return barXScale(d[0]); })
                               .attr("id", function(d) { return d[1] })
                               .attr("fill", function(d) {
                                    var color;
                                    var countMatch = 0;
                                    var allNodes = [];
                        
                                    circles.forEach(function(row, rowIndex) {
                                        row.forEach(function(circle, circleIndex) {
                                            if ((circle.__data__).length == 4) {
                                                allNodes.push(circle.__data__);
                                            }
                                        });
                                    });
                            
                                    allNodes.forEach(function(bars) {
                                        bars[3].forEach(function(bar) {
                                            if (bar.indexOf(d[1]) >= 0) {
                                                countMatch++;
                                            }
                                        });
                                    });
                                    
                                    if (countMatch == allNodes.length) {
                                        color = "RosyBrown";
                                    } else {
                                        color = "Grey";
                                    }
                       
                                    if (isMergedNode) {
                                        var previousNodesWords = [];
                                        var wordMatch = 0;
                                        var indexMatch = 0;
                                        
                                        for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                            var wordsArray = previousColorNodes[i].__data__[3];
                                            wordsArray.forEach(function(item) {
                                                if (item.indexOf(d[1]) >= 0) {
                                                    wordMatch += 1;
                                                    indexMatch = i;
                                                }
                                            });
                                        }
                                        if (wordMatch == 1) {
                                                color = previousColorNodes[indexMatch].getAttribute("click-fill-color");
                                        }
                                        if (wordMatch > 1) {
                                            color = "Tan";
                                        }
                                    }
                                    return color;
                                })
                                .on("mouseover", function(d) {
                                    if (isMergedNode) {
                                        tempColor = document.getElementById(d[1]).getAttribute("fill");
                                        for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                            var wordsArray = previousColorNodes[i].__data__[3];
                                            var match = false;
                                            wordsArray.forEach(function(item) {
                                                if (item.indexOf(d[1]) >= 0) {
                                                    match = true;
                                                }
                                            });
                                            if (match) {
                                                previousColorNodes[i].style.fill = "#dd4477";
                                            }
                                        }
                                        document.getElementById(d[1]).style.fill = "#dd4477";
                                    }
                                })
                                .on("mouseout", function(d) {
                                    if (isMergedNode) {
                                        document.getElementById(d[1]).style.fill = tempColor;
                                        for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                            var wordsArray = previousColorNodes[i].__data__[3];
                                            var match = false;
                                            wordsArray.forEach(function(item) {
                                                if (item.indexOf(d[1]) >= 0) {
                                                    match = true;
                                                }
                                            });
                                            if (match) {
                                                previousColorNodes[i].style.fill = previousColorNodes[i]
                                                                                   .getAttribute("click-fill-color");
                                            }
                                        }
                                        tempColor = null;
                                    }
                                });
                        
                    singlePopup.selectAll("text")
                               .data(singleNodeBarDataset)
                               .enter()
                               .append("text")
                               .attr("x", "0")
                               .attr("y", function(d, i) { return i * 25; })
                               .attr("dx", "5")
                               .attr("dy", "15")
                               .style("fill", "SeaShell")
                               .text(function(d) { return d[1]; })
                               .attr("class", function(d) { return "text" + d[1]; })
                               .on("mouseover", function(d) {
                                    if (isMergedNode) {
                                        tempColor = document.getElementById(d[1]).getAttribute("fill");
                                        for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                            var wordsArray = previousColorNodes[i].__data__[3];
                                            var match = false;
                                            wordsArray.forEach(function(item) {
                                                if (item.indexOf(d[1]) >= 0) {
                                                    match = true;
                                                }
                                            });
                                            if (match) {
                                                previousColorNodes[i].style.fill = "#dd4477";
                                            }
                                        }
                                        document.getElementById(d[1]).style.fill = "#dd4477";
                                    }
                               })
                               .on("mouseout", function(d) {
                                    if (isMergedNode) {
                                        document.getElementById(d[1]).style.fill = tempColor;
                                        for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                                            var wordsArray = previousColorNodes[i].__data__[3];
                                            var match = false;
                                            wordsArray.forEach(function(item) {
                                                if (item.indexOf(d[1]) >= 0) {
                                                    match = true;
                                                }
                                            });
                                            if (match) {
                                                previousColorNodes[i].style.fill = previousColorNodes[i]
                                                                                   .getAttribute("click-fill-color");
                                            }
                                        }
                                        tempColor = null;
                                    }  
                               });
                 })
                .on("mouseover", function(d) {
                    mousePosition = d3.mouse(canvas.node()); 
                    tooltip.transition()
                           .duration(200)
                           .style("opacity", .9);
                    tooltip.html("Number of Tickets: " + d[2])
                           .style("left", (mousePosition[0] + margin.left) + "px")
                           .style("top", (mousePosition[1] - 30) + "px")
                           .style("color", "SeaShell")
                           .style("padding", "5px");
                })
                 .on("mouseout", function(d) {
                    tooltip.transition()
                           .duration(500)
                           .style("opacity", 0);
                 });

            text
                .attr("y", index * y_margin + 18)
                .attr("x", function(d, i) { return xScale(d[0]) - 15; })
                .attr("class","value")
                .text(function(d){ return d[2]; })
                .style("fill", "66aa00")
                .style("display","none");

            g.append("text")
             .attr("y", index * y_margin + 25)
             .attr("x", xAxis_length + 35)
             .attr("class","label")
             .text(truncate(item['words'],10))
             .style("fill", "Grey")
             .on("mouseover", mouseover_all)
             .on("mouseout", mouseout_all)
             .on("click", click_all)
                 
        });

        function mouseover_all(p) {
            var g = d3.select(this).node().parentNode;
            d3.select(g).selectAll("circle").style("display","none");
            d3.select(g).selectAll("text.value").style("display","block");
        }

        function mouseout_all(p) {
            var g = d3.select(this).node().parentNode;
            d3.select(g).selectAll("circle").style("display","block");
            d3.select(g).selectAll("text.value").style("display","none");
        }
        
        function click_all(p) {
            
            $("#popup").remove();

            var isMergedNode = false;
            var previousPopup = document.getElementById("single-node-popup");
                
            if (previousPopup) {
                for (var i = previousColorNodes.length - 1; i >= 0; i--) {
                    previousColorNodes[i].style.fill = previousColorNodes[i].getAttribute("fill-color");
                }
                previousColorNodes.length = 0;
                previousColorNodes = [];
                previousPopup.remove();
            }
                
            mousePosition = d3.mouse(canvas.node()); 
            
            //create a set of word boxes
            var popup = canvas.append("div")
                              .attr("id", "popup")
                              .style("position", "absolute")
                              .style("left", (mousePosition[0] + 100) + "px")
                              .style("top", (mousePosition[1] + 10) + "px")
                              .style("background-color", "Rgba(0,0,0,0.8)")
                              .style("padding", "10px")
                              .on("click", function() {
                                  $("#popup").remove(); 
                              });
            
            popup.append("p") 
                     .style("font-size", "12px") 
                     .style("text-decoration", "underline")
                     .style("color", "SeaShell")
                     .text("Word Frequency");
            
            var g = d3.select(this).node().parentNode;
                    
            d3.select(g).selectAll("circle").forEach(function(row, rowIndex) {
                    
                var barChartDataset = [];
                    
                row.forEach(function(circle, circleIndex) {
                    if ((circle.__data__).length == 4) {
                        barChartDataset.push(circle.__data__);
                    }
                    
                });
                
                barChartDataset.forEach(function(circle, circleIndex) {

                    popup.style("width", (barChartDataset.length * 100 + 20) + "px");
                    popup.append("div")
                         .attr("id", "div" + circleIndex)
                         .style("width", "100px")
                         .style("display", "inline-block")
                         .append("p")
                         .text(circle[0])
                         .style("color", "SeaShell")
                         .style("font-size", "12px") ;

                    var singlePopup = d3.select("#div" + circleIndex)
                                        .append("svg")
                                        .style("width", "100px")
                                        .style("height", "245px");

                    var singleNodeDataset = circle[3].sort(function(a, b) {
                        if (a[0] < b[0]) {
                            return 1;
                        }
                        if (a[0] > b[0]) {
                            return -1;
                        }
                        return 0;
                    });

                    var barXScale = d3.scale.linear()
                                            .domain([0, d3.max(singleNodeDataset, function(d) {
                                                return d[0];
                                            })])
                                            .range([0,90]);

                    singlePopup.selectAll("rect")
                               .data(singleNodeDataset)
                               .enter()
                               .append("rect")
                               .attr("x", 0)
                               .attr("y", function(d, i) { return (i * 25); })
                               .attr("height", "20")
                               .attr("width", function(d) { return barXScale(d[0]); })
                               .attr("class", function(d) { return d[1] })
                               .attr("fill", function(d) {
                        
                                    var countMatch = 0;

                                    barChartDataset.forEach(function(bars) {
                                        bars[3].forEach(function(bar) {
                                            if (bar.indexOf(d[1]) >= 0) {
                                                countMatch++;
                                            }
                                        });
                                    });

                                    if (countMatch == barChartDataset.length) {
                                        return "RosyBrown";
                                    } else {
                                        return "Grey";
                                    }
                                })
                                .on("mouseover", function(d) {

                                    var popupContainer = document.getElementById("popup");
                                    var children;

                                    var rects = popupContainer.getElementsByTagName("rect");
                                    for (var i = 0; i < rects.length; i++) {
                                        rects[i].addEventListener("mouseover", function() {
                                            children = d3.select(this).node().parentNode.parentNode.parentNode
                                                             .getElementsByClassName(d3.select(this).node()
                                                                                       .className.baseVal);

                                            for (var j = 0; j < children.length; j++) {
                                                tempColor = children[j].style.fill;
                                                children[j].style.fill = "#dd4477";
                                            }
                                        })
                                        rects[i].addEventListener("mouseout", function() {
                                            children = d3.select(this).node().parentNode.parentNode.parentNode
                                                             .getElementsByClassName(d3.select(this).node()
                                                                                       .className.baseVal);

                                            for (var j = 0; j < children.length; j++) {
                                                children[j].style.fill = tempColor;
                                                tempColor = null;
                                            }
                                        })
                                    }
                            
                                    var texts = popupContainer.getElementsByTagName("text");
                                    for (var i = 0; i < texts.length; i++) {
                                        texts[i].addEventListener("mouseover", function() {
                                            children = d3.select(this).node().parentNode.parentNode.parentNode
                                                             .getElementsByClassName(d3.select(this).node().className
                                                                                       .baseVal.substring(4));

                                            for (var j = 0; j < children.length; j++) {
                                                tempColor = children[j].style.fill;
                                                children[j].style.fill = "#dd4477";
                                            }
                                        })
                                        texts[i].addEventListener("mouseout", function() {
                                            children = d3.select(this).node().parentNode.parentNode.parentNode
                                                             .getElementsByClassName(d3.select(this).node().className
                                                                                       .baseVal.substring(4));

                                            for (var j = 0; j < children.length; j++) {
                                                children[j].style.fill = tempColor;
                                                tempColor = null;
                                            }
                                        })
                                    }
                                });

                    singlePopup.selectAll("text")
                               .data(singleNodeDataset)
                               .enter()
                               .append("text")
                               .attr("x", "0")
                               .attr("y", function(d, i) { return i * 25; })
                               .attr("dx", "5")
                               .attr("dy", "15")
                               .style("fill", "SeaShell")
                               .text(function(d) { return d[1]; })
                               .attr("class", function(d) { return "text" + d[1]; }); 
                });
            });
        }
    });
});

<IPython.core.display.Javascript object>